# Market Value Prediction Model
---

This model will use the usa-real-estate dataset of data scraped from `realtor.com`.

## Imports
---

In [11]:
import pandas as pd;
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import utils;
from sklearn.preprocessing import LabelEncoder;
from sklearn.model_selection import train_test_split;
import torch;
from torch import nn;
from torch.utils.data import DataLoader, Dataset;
from tqdm.auto import tqdm;
from typing import Tuple
import torch.onnx;

DEVICE: str = "cuda" if torch.cuda.is_available() else "cpu";
DEVICE

'cuda'

## Loading Data
---

In [2]:
df = pd.read_csv("realtor-data.zip.csv");
df.head(5), df.columns

(   brokered_by    status     price  bed  bath  acre_lot     street  \
 0     103378.0  for_sale  105000.0  3.0   2.0      0.12  1962661.0   
 1      52707.0  for_sale   80000.0  4.0   2.0      0.08  1902874.0   
 2     103379.0  for_sale   67000.0  2.0   1.0      0.15  1404990.0   
 3      31239.0  for_sale  145000.0  4.0   2.0      0.10  1947675.0   
 4      34632.0  for_sale   65000.0  6.0   2.0      0.05   331151.0   
 
          city        state  zip_code  house_size prev_sold_date  
 0    Adjuntas  Puerto Rico     601.0       920.0            NaN  
 1    Adjuntas  Puerto Rico     601.0      1527.0            NaN  
 2  Juana Diaz  Puerto Rico     795.0       748.0            NaN  
 3       Ponce  Puerto Rico     731.0      1800.0            NaN  
 4    Mayaguez  Puerto Rico     680.0         NaN            NaN  ,
 Index(['brokered_by', 'status', 'price', 'bed', 'bath', 'acre_lot', 'street',
        'city', 'state', 'zip_code', 'house_size', 'prev_sold_date'],
       dtype='object

## Data Pre-Processing
---

in this section, I have a lot of bad data, I will have two renditions of my dataframe, one with missing data replaced by the median, one replaced by the mean, and one with missing data fully removed. They will be placed in `whole_df`.

In [3]:
dataset_outlier_mode: str = "median"; # median | stripped | mean

# Only operating on listings for sale
whole_df = df[df['status'] != "sold"];

# Street encoded value cannot be reproduced when given new data.
# prev_sold_date useless, not a time series problem.
whole_df = whole_df.drop(columns=['street', 'status', 'prev_sold_date', 'city', 'brokered_by'], axis=1);

# Before Cleaning
utils.dataFrameStatus(whole_df)

if dataset_outlier_mode == "mean":
    whole_df['bed'] = whole_df['bed'].fillna(whole_df['bed'].mean());
    whole_df['bath'] = whole_df['bath'].fillna(whole_df['bath'].mean());
    whole_df['acre_lot'] = whole_df['acre_lot'].fillna(whole_df['acre_lot'].mean());
    whole_df['house_size'] = whole_df['house_size'].fillna(whole_df['house_size'].mean());
elif dataset_outlier_mode == "median":
    whole_df['bed'] = whole_df['bed'].fillna(whole_df['bed'].median());
    whole_df['bath'] = whole_df['bath'].fillna(whole_df['bath'].median());
    whole_df['acre_lot'] = whole_df['acre_lot'].fillna(whole_df['acre_lot'].median());
    whole_df['house_size'] = whole_df['house_size'].fillna(whole_df['house_size'].median());

whole_df = whole_df.dropna();

state_map = {
    'Puerto Rico': 'PR',
    'Virgin Islands': 'VI',
    'Massachusetts': 'MA',
    'Connecticut': 'CT',
    'New Jersey': 'NJ',
    'New York': 'NY',
    'New Hampshire': 'NH',
    'Vermont': 'VT',
    'Rhode Island': 'RI',
    'Wyoming': 'WY',
    'Maine': 'ME',
    'Pennsylvania': 'PA',
    'West Virginia': 'WV',
    'Delaware': 'DE',
    'Ohio': 'OH',
    'Maryland': 'MD',
    'Virginia': 'VA',
    'Colorado': 'CO',
    'District of Columbia': 'DC',
    'North Carolina': 'NC',
    'Kentucky': 'KY',
    'South Carolina': 'SC',
    'Tennessee': 'TN',
    'Georgia': 'GA',
    'Alabama': 'AL',
    'Florida': 'FL',
    'Mississippi': 'MS',
    'Texas': 'TX',
    'Missouri': 'MO',
    'Arkansas': 'AR',
    'Louisiana': 'LA',
    'Indiana': 'IN',
    'Illinois': 'IL',
    'Michigan': 'MI',
    'Wisconsin': 'WI',
    'Iowa': 'IA',
    'Minnesota': 'MN',
    'South Dakota': 'SD',
    'Nebraska': 'NE',
    'North Dakota': 'ND',
    'Montana': 'MT',
    'Idaho': 'ID',
    'Kansas': 'KS',
    'Oklahoma': 'OK',
    'New Mexico': 'NM',
    'Utah': 'UT',
    'Nevada': 'NV',
    'Washington': 'WA',
    'Oregon': 'OR',
    'Arizona': 'AZ',
    'California': 'CA',
    'Hawaii': 'HI',
    'Guam': 'GU',
    'Alaska': 'AK'
}

whole_df['state'] = whole_df['state'].map(state_map);

le = LabelEncoder();
whole_df['state'] = le.fit_transform(whole_df['state']);

# After Cleaning
print("\nAfter Cleaning\n");
utils.dataFrameStatus(whole_df)
len(whole_df), whole_df.head(5)


LOOKING AT DF size(1414373)
+====

| FEATURE price :
| NULL COUNT -> 1296 | NaN COUNT -> 1296 | NONE COUNT -> 0
| TOTAL MISSING VALUES 2592 -> 0.18%
---
| FEATURE bed :
| NULL COUNT -> 411343 | NaN COUNT -> 411343 | NONE COUNT -> 0
| TOTAL MISSING VALUES 822686 -> 58.17%
---
| FEATURE bath :
| NULL COUNT -> 433842 | NaN COUNT -> 433842 | NONE COUNT -> 0
| TOTAL MISSING VALUES 867684 -> 61.35%
---
| FEATURE acre_lot :
| NULL COUNT -> 217967 | NaN COUNT -> 217967 | NONE COUNT -> 0
| TOTAL MISSING VALUES 435934 -> 30.82%
---
| FEATURE state :
| NULL COUNT -> 8 | NaN COUNT -> 8 | NONE COUNT -> 0
| TOTAL MISSING VALUES 16 -> 0.00%
---
| FEATURE zip_code :
| NULL COUNT -> 287 | NaN COUNT -> 287 | NONE COUNT -> 0
| TOTAL MISSING VALUES 574 -> 0.04%
---
| FEATURE house_size :
| NULL COUNT -> 464774 | NaN COUNT -> 464774 | NONE COUNT -> 0
| TOTAL MISSING VALUES 929548 -> 65.72%
---

+====


After Cleaning

LOOKING AT DF size(1412785)
+====

| FEATURE price CONTAINS NO EMPTY VALUES ++
---
| FEAT

(1412785,
       price  bed  bath  acre_lot  state  zip_code  house_size
 0  105000.0  3.0   2.0      0.12     40     601.0       920.0
 1   80000.0  4.0   2.0      0.08     40     601.0      1527.0
 2   67000.0  2.0   1.0      0.15     40     795.0       748.0
 3  145000.0  4.0   2.0      0.10     40     731.0      1800.0
 4   65000.0  6.0   2.0      0.05     40     680.0      1822.0)

## Splitting Data
---

In [4]:
BATCH_SIZE: int = 32;

class CustomDataset(Dataset):
    def __init__(self, df: pd.DataFrame) -> None:
        super().__init__()
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        col_names = [col for col in self.df.columns if col != "price"]
        features = torch.tensor(self.df.iloc[index][col_names].values, dtype=torch.float32)
        label = torch.tensor(self.df.iloc[index]['price'], dtype=torch.float32)

        return features, label;


train_df, test_df = train_test_split(whole_df, test_size=0.2, train_size=0.8);

train_dataset = CustomDataset(train_df);
test_dataset = CustomDataset(test_df);

train_loader = DataLoader(
    dataset=train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True);

test_loader = DataLoader(
    dataset=test_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False);

next(iter(train_loader)), next(iter(test_loader))

([tensor([[4.0000e+00, 3.0000e+00, 2.0000e+01, 3.7000e+01, 7.3857e+04, 2.1250e+03],
          [3.0000e+00, 3.0000e+00, 3.3000e-01, 4.5000e+01, 7.7384e+04, 3.0800e+03],
          [4.0000e+00, 4.0000e+00, 1.0700e+00, 1.5000e+01, 6.0014e+04, 4.2380e+03],
          [4.0000e+00, 2.0000e+00, 4.1000e-01, 1.3000e+01, 5.2803e+04, 4.1780e+03],
          [3.0000e+00, 2.0000e+00, 1.3000e-01, 1.4000e+01, 8.3815e+04, 1.3970e+03],
          [3.0000e+00, 3.0000e+00, 3.3000e-01, 4.3000e+01, 5.7032e+04, 1.4460e+03],
          [3.0000e+00, 2.0000e+00, 1.4400e+01, 3.8000e+01, 9.7405e+04, 1.8220e+03],
          [3.0000e+00, 2.0000e+00, 1.0000e+00, 2.3000e+01, 4.9637e+04, 1.8220e+03],
          [3.0000e+00, 2.0000e+00, 9.8000e+01, 1.0000e+01, 3.1036e+04, 1.8220e+03],
          [3.0000e+00, 2.0000e+00, 8.0000e-02, 4.0000e+00, 9.4123e+04, 1.8220e+03],
          [3.0000e+00, 2.0000e+00, 2.1500e+00, 4.2000e+01, 2.9906e+04, 1.8220e+03],
          [4.0000e+00, 3.0000e+00, 2.1500e+00, 4.7000e+01, 2.2972e+04, 1.610

## Model Architecture
---

In [5]:
class LinearBaseline(nn.Module):
    def __init__(self, input_features, num_blocks, hidden_units=64, dropout=0.3) -> None:
        super().__init__();

        self.num_blocks = num_blocks;

        self.input = nn.Linear(in_features=input_features, out_features=hidden_units);

        self.block = nn.Sequential(
            nn.Linear(in_features=hidden_units, out_features=hidden_units),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_units),
            nn.Dropout(dropout)
        );
    
        self.classifier = nn.Linear(in_features=hidden_units, out_features=1);

    def forward(self, x: torch.Tensor):
        x = self.input(x);
        for _ in range(self.num_blocks):
            x = self.block(x);

        return self.classifier(x);

model = LinearBaseline(hidden_units=256,
                       dropout=0.3,
                       input_features=len(next(iter(train_loader))[0][0]),
                       num_blocks=3).to(DEVICE);

model.state_dict()

OrderedDict([('input.weight',
              tensor([[ 0.1864, -0.3295, -0.3936, -0.0234,  0.1892,  0.1117],
                      [-0.3620,  0.1725,  0.3694,  0.0908,  0.2562,  0.1671],
                      [-0.3317, -0.2761, -0.0457, -0.1734,  0.0671,  0.3174],
                      ...,
                      [ 0.1294,  0.1375, -0.2727, -0.3260,  0.3195,  0.3195],
                      [ 0.3555,  0.3024,  0.0314, -0.0733,  0.1427, -0.1676],
                      [-0.0467, -0.3274,  0.0170,  0.0425,  0.0430,  0.0836]],
                     device='cuda:0')),
             ('input.bias',
              tensor([ 0.1869,  0.1592, -0.0404,  0.1091, -0.3488, -0.2420,  0.1016, -0.2041,
                       0.2177,  0.3118, -0.0114,  0.0813,  0.2784, -0.0549,  0.3290, -0.1638,
                      -0.1380, -0.0228, -0.3391, -0.1975,  0.3225,  0.2774,  0.1266,  0.4001,
                       0.0130, -0.3547, -0.0322,  0.1195, -0.0232,  0.0562, -0.1198, -0.1237,
                       0.3163,

# Train Step
---

In [14]:
def train_step(
    model: LinearBaseline,
    loader: DataLoader,
    loss_fn: nn.Module,
    optim: torch.optim.Optimizer) -> Tuple[float, float]:

    running_loss = 0.0;
    running_acc = 0.0;
    total_samples = 0;
    
    for X, y in tqdm(loader):
        X: torch.Tensor = X.to(DEVICE);
        y: torch.Tensor = y.to(DEVICE);

        # Ensure model output matches target shape
        outputs: torch.Tensor = model(X).squeeze(-1);

        if outputs.shape == y.shape:
            loss = loss_fn(outputs, y);
            optim.zero_grad();
            loss.backward();
            optim.step();
            running_loss += loss.item() * X.size(0);
            running_acc += utils.batchAccuracy(outputs, y) * X.size(0);
            total_samples += X.size(0);

    
    avg_loss = running_loss / total_samples if total_samples > 0 else 0.0
    avg_acc = running_acc / total_samples if total_samples > 0 else 0.0
    
    return avg_loss, avg_acc


## Test Step
---

In [15]:
def test_step(
    model: LinearBaseline,
    loader: DataLoader,
    loss_fn: nn.Module) -> Tuple[float, float]:
    
    running_loss = 0.0;
    running_acc = 0.0;
    total_samples = 0;
    
    for X, y in tqdm(loader):
        X: torch.Tensor = X.to(DEVICE);
        y: torch.Tensor = y.to(DEVICE);

        outputs: torch.Tensor = model(X).squeeze(-1);

        if outputs.shape == y.shape:
            loss = loss_fn(outputs, y)
            running_loss += loss.item() * X.size(0)  
            running_acc += utils.batchAccuracy(outputs, y) * X.size(0) 
            total_samples += X.size(0)
    
    avg_loss = running_loss / total_samples if total_samples > 0 else 0.0
    avg_acc = running_acc / total_samples if total_samples > 0 else 0.0
    
    return avg_loss, avg_acc

## Training Model
---

In [16]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2);
loss_fn = torch.nn.MSELoss();
EPOCHS = 25;

# loop over the dataset multiple times
for epoch in tqdm(range(EPOCHS)):
    
    model.train();
    train_loss, train_acc = train_step(
        loader=train_loader,
        loss_fn=loss_fn,
        model=model,
        optim=optimizer,
    );

    with torch.inference_mode():
        model.eval();
        test_loss, test_acc = test_step(
            loader=test_loader,
            loss_fn=loss_fn,
            model=model,
        );

    print(f"Epoch {epoch+1}/{EPOCHS}");
    print(f"Training Accuracy: {train_acc:.2f}%, Test Accuracy: {test_acc:.2f}%");
    print(f"Training Loss: {train_loss:.4f}%, Test Loss: {test_loss:.4f}%");

print('Finished Training')

  4%|▍         | 1/25 [12:24<4:57:36, 744.04s/it]

Epoch 1/25
Training Accuracy: 0.00%, Test Accuracy: 0.00%
Training Loss: 2747516746498.4800%, Test Loss: 154952013531812822480060416.0000%


  8%|▊         | 2/25 [24:41<4:43:41, 740.05s/it]

Epoch 2/25
Training Accuracy: 0.00%, Test Accuracy: 0.00%
Training Loss: 2732628073961.0327%, Test Loss: inf%


 12%|█▏        | 3/25 [37:03<4:31:44, 741.11s/it]

...

Epoch 23/25
Training Accuracy: 0.00%, Test Accuracy: 0.00%
Training Loss: 2663226798559.8384%, Test Loss: 9162036833022573568.0000%


 96%|█████████▌| 24/25 [4:43:03<11:20, 680.11s/it]

Epoch 24/25
Training Accuracy: 0.00%, Test Accuracy: 0.00%
Training Loss: 2661487537456.0176%, Test Loss: 170876709468083680.0000%


100%|██████████| 25/25 [4:54:02<00:00, 705.69s/it]

Epoch 25/25
Training Accuracy: 0.00%, Test Accuracy: 0.00%
Training Loss: 2658720516697.4976%, Test Loss: 18481937625453801472.0000%
Finished Training


## Saving the Model
---

In [19]:
dummy_input = torch.randn(1, 6).to(DEVICE);
torch.onnx.export(model,
                  dummy_input,
                  f"{dataset_outlier_mode}-model.onnx",
                  opset_version=11)